# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character：角色姓名。
- role：演职员类型，演员或导演。

# 读取数据

In [1]:
import pandas as pd

In [2]:
titles = pd.read_csv("titles.csv")
credits = pd.read_csv("credits.csv")
credits.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


In [3]:
titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


# 评估数据及清洗数据

In [4]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


”character“一项有缺失值，但并不影响对评分的分析

In [5]:
credits.sample(10)

,person_id,id,name,character,role
44920,18135,tm368699,Phillip DeVona,Calvin Claytor (uncredited),ACTOR
76849,1181245,tm1029279,Lauren Daigle,NaN,ACTOR
55995,88185,tm897170,Stéphane Caillard,Chloé Diallo,ACTOR
66285,103556,ts286731,Rie Murakawa,Najimi Osana (voice),ACTOR
28109,1681389,tm315344,John Krpan,LA Player,ACTOR
21974,108467,tm196125,A.J. Ackleson,Hipster Turkey Man,ACTOR
56766,2429724,tm496648,Nicolas Gerout,"Fred, Samia's boyfriend",ACTOR
34795,108073,ts131579,Tibo Vandenborre,De Spanjol,ACTOR
5458,44346,ts30311,Avan Jogia,Beck Oliver,ACTOR
53752,648215,tm454824,Arkadiy Koval,Nikolai Nikolaevich's governor,ACTOR


无结构性问题，下面看是否有内容性问题

In [6]:
credits.duplicated().sum()

0

无重复性数据

In [7]:
credits.value_counts(subset = ["role"])

role    
ACTOR       73251
DIRECTOR     4550
Name: count, dtype: int64

无不一致数据

由于是演员表，无需评估无效或错误数据

In [8]:
cleaned_credits = credits.copy()

对person_id一项，转换成字符串

In [9]:
cleaned_credits["person_id"] = cleaned_credits["person_id"].astype("str")
cleaned_credits["person_id"]

0           3748
1          14658
2           7064
3           3739
4          48933
          ...   
77796     736339
77797     399499
77798     373198
77799     378132
77800    1950416
Name: person_id, Length: 77801, dtype: object

下面看titles一表

In [10]:
titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5850 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5850 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5850 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5850 non-null   int64  
 7   genres                5850 non-null   object 
 8   production_countries  5850 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5447 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(5), int64(

In [11]:
titles.sample(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
2976,tm350945,Posesif,MOVIE,A student diver risks her scholastic future an...,2017,NaN,102,"['drama', 'comedy', 'romance']",['ID'],NaN,tt7541708,7.4,920.0,4.901,7.8
1425,tm240895,Strange Weather,MOVIE,"Years after her son's suicide, a woman longs t...",2016,R,95,['drama'],['US'],NaN,tt5205210,6.0,1499.0,9.634,5.9
2907,tm325650,King of Peking,MOVIE,When home entertainment enters the market in 9...,2017,NaN,88,"['drama', 'comedy']","['AU', 'US', 'CN']",NaN,tt6406976,6.4,368.0,1.549,6.2
3177,tm441232,Someone Great,MOVIE,An aspiring music journalist lands her dream j...,2019,R,92,"['romance', 'comedy']",['US'],NaN,tt8075260,6.1,22277.0,14.238,6.4
3274,tm411750,Doom: Annihilation,MOVIE,A group of UAC Marines responds to a distress ...,2019,R,96,"['action', 'scifi', 'thriller', 'horror']",['US'],NaN,tt8328716,3.7,9281.0,29.955,4.9
5308,tm879685,The Perfect Family,MOVIE,Lucia's achievement of a perfect family is thr...,2021,NaN,110,['comedy'],['ES'],NaN,tt11924738,5.3,1931.0,NaN,NaN
5541,tm810657,Uppena,MOVIE,Aasi has been in love with Bebamma since their...,2021,NaN,147,['drama'],['IN'],NaN,NaN,NaN,NaN,4.709,6.3
857,ts14583,BoJack Horseman,SHOW,Meet the most beloved sitcom horse of the 90s ...,2014,TV-MA,26,"['comedy', 'drama', 'animation']",['US'],6.0,tt3398228,8.8,148127.0,92.956,8.6
2944,tm350116,B. A. Pass 2,MOVIE,"B. A. Pass 2 is the story of a young girl, who...",2017,NaN,136,['drama'],['IN'],NaN,tt8581230,2.2,483.0,3.448,7.0
5548,tm855340,Thalaivii,MOVIE,The biopic charts the life of actor-turned-chi...,2021,G,153,['drama'],['IN'],NaN,tt10152736,5.9,30621.0,2.301,5.0


先看结构性问题，可以发现在genres、production_countries两列中，存在含有多个值的单元格，下面对其进行处理

In [12]:
cleaned_titles = titles.copy()

由于genres一列的数据类型是object，需要先将其转换成列表

In [13]:
cleaned_titles["genres"] = cleaned_titles["genres"].apply(lambda x: eval(x))
cleaned_titles["genres"][0]

['documentation']

使用explode方法，拆分行

In [14]:
cleaned_titles = cleaned_titles.explode("genres")

In [15]:
cleaned_titles.sample(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1857,ts82560,Dark Tourist,SHOW,"The definition of ""tourism"" is redefined as Ne...",2018,TV-MA,41,documentation,['NZ'],1.0,tt8725166,7.5,7233.0,6.668,6.500
1219,ts52838,3%,SHOW,In a future where the elite inhabit an island ...,2016,TV-MA,46,action,['BR'],4.0,tt4922804,7.3,25452.0,29.033,7.200
366,ts30288,Mobile Suit Gundam Unicorn,SHOW,"The series begins in U.C. 0001, at the very be...",2010,TV-14,36,war,['JP'],2.0,tt1587790,7.8,1291.0,10.936,7.900
152,tm117488,The Next Karate Kid,MOVIE,"Mr. Miyagi decides to take Julie, a troubled t...",1994,PG,107,romance,['US'],NaN,tt0110657,4.5,30422.0,20.185,5.318
353,tm103095,Ip Man 2,MOVIE,Having defeated the best fighters of the Imper...,2010,R,108,history,"['HK', 'CN']",NaN,tt1386932,7.5,104500.0,4.516,7.500
1244,ts3844,Aquarius,SHOW,"In the late 1960s, a Los Angeles police sergea...",2015,TV-14,43,crime,['US'],2.0,tt3768572,7.0,12279.0,13.111,6.800
280,tm29737,Rambo,MOVIE,When governments fail to act on behalf of capt...,2008,R,92,thriller,"['DE', 'US']",NaN,tt0462499,7.0,230333.0,72.224,6.600
1763,ts58155,Alias Grace,SHOW,"Based on the true story of Grace Marks, a hous...",2017,TV-MA,44,history,['CA'],1.0,tt1034007,7.7,32177.0,13.586,7.300
738,tm177044,Naruto Shippuden the Movie: Blood Prison,MOVIE,After his capture for attempted assassination ...,2011,PG-13,108,animation,['JP'],NaN,tt1999167,7.1,5104.0,184.852,7.200
2562,ts83204,Million Pound Menu,SHOW,Participants try to convince a jury that their...,2018,NaN,60,reality,['GB'],2.0,tt8488494,6.6,565.0,4.744,6.200


同理，对production_countries一列进行相同的操作

In [16]:
cleaned_titles["production_countries"] = cleaned_titles["production_countries"].apply(lambda x: eval(x))

In [17]:
cleaned_titles = cleaned_titles.explode("production_countries")
cleaned_titles.sample(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1023,tm175954,Aziz Ansari: Buried Alive,MOVIE,"Standup comedian Aziz Ansari (""Parks and Recre...",2013,NaN,79,documentation,US,NaN,tt2836450,7.2,3998.0,5.417,7.0
1581,tm235227,Bill Hicks: Reflections,MOVIE,Comedy Dynamics sits down with Bill Hicks’ bro...,2015,NaN,34,documentation,US,NaN,NaN,NaN,NaN,2.465,6.7
1963,tm211554,Beyond Skyline,MOVIE,Detective Mark Corley storms his way onto an a...,2017,R,105,action,SG,NaN,tt6856700,5.3,21834.0,10.535,5.2
693,tm176507,Jackass 3.5,MOVIE,Johnny Knoxville of 'Jackass' releases unused ...,2011,NaN,85,documentation,US,NaN,NaN,NaN,NaN,19.565,6.3
2748,tm279017,"Michael Bolton's Big, Sexy Valentine's Day Spe...",MOVIE,After Santa tells Michael Bolton that he needs...,2017,NaN,54,music,US,NaN,tt6343706,6.8,2126.0,4.616,6.4
3931,ts90280,Yankee,SHOW,"On the run from the police, an Arizona man cro...",2019,TV-MA,38,crime,MX,1.0,tt9046782,5.8,701.0,8.380,6.6
5843,tm1097142,My Bride,MOVIE,The story follows a young man and woman who go...,2021,NaN,93,comedy,EG,NaN,tt14216488,5.0,327.0,2.545,5.3
845,tm176864,Rush,MOVIE,A biographical drama centered on the rivalry b...,2013,R,123,sport,DE,NaN,tt1979320,8.1,471586.0,29.759,7.7
4768,ts222522,Spriggan,SHOW,An ancient civilization's relics on Earth hold...,2022,TV-MA,46,thriller,JP,1.0,tt10011306,7.0,664.0,16.710,6.9
420,tm33981,Bon Cop Bad Cop,MOVIE,When the body of the executive of hockey Benoi...,2006,PG-13,116,scifi,CA,NaN,tt0443521,6.5,61.0,7.956,6.4


结构性问题处理完毕，下面看是否有内容性问题

In [18]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17818 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17818 non-null  object 
 1   title                 17817 non-null  object 
 2   type                  17818 non-null  object 
 3   description           17790 non-null  object 
 4   release_year          17818 non-null  int64  
 5   age_certification     10889 non-null  object 
 6   runtime               17818 non-null  int64  
 7   genres                17755 non-null  object 
 8   production_countries  17439 non-null  object 
 9   seasons               6224 non-null   float64
 10  imdb_id               17116 non-null  object 
 11  imdb_score            16976 non-null  float64
 12  imdb_votes            16945 non-null  float64
 13  tmdb_popularity       17663 non-null  float64
 14  tmdb_score            17241 non-null  float64
dtypes: float64(5), int64(2), 

将release_year转换成datetime

In [19]:
cleaned_titles["release_year"] = pd.to_datetime(cleaned_titles["release_year"], format='%Y')
cleaned_titles["release_year"]

0      1945-01-01
1      1976-01-01
1      1976-01-01
2      1972-01-01
2      1972-01-01
          ...    
5847   2021-01-01
5848   2021-01-01
5849   2021-01-01
5849   2021-01-01
5849   2021-01-01
Name: release_year, Length: 17818, dtype: datetime64[ns]

不难发现，有许多列存在缺失数据的情况，但是genres和imdb_score缺失会影响到我们的分析过程

In [20]:
cleaned_titles[cleaned_titles["genres"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
212,tm255589,One Last Shot,MOVIE,"In this low-budget short film, two best buddie...",1998-01-01,NaN,30,NaN,NaN,NaN,NaN,NaN,NaN,1.890,5.2
619,tm341561,Like Twenty Impossibles,MOVIE,Occupied Palestine: A serene landscape now poc...,2003-01-01,NaN,16,NaN,PS,NaN,NaN,NaN,NaN,0.812,6.5
632,ts86241,Le Robe De Mariage Des Cieux,SHOW,It was with much difficulty that Ai Qing was a...,2004-01-01,TV-MA,63,NaN,NaN,1.0,NaN,NaN,NaN,0.600,NaN
636,tm404676,To and from New York,MOVIE,"While covering a story in New York City, a Sea...",2006-01-01,NaN,82,NaN,US,NaN,NaN,NaN,NaN,1.401,5.8
637,tm89054,Osuofia in London 2,MOVIE,Osuofia return to his Nigerian village with a ...,2004-01-01,NaN,72,NaN,XX,NaN,NaN,NaN,NaN,1.091,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,tm1040959,The Circle: The Afterparty,MOVIE,Stars of The Circle drop by to discuss Season ...,2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,1.882,10.0
5802,ts302434,Plastic Cup Boyz: Laughing My Mask Off!,SHOW,Comedy collective The Plastic Cup Boyz pour ou...,2021-01-01,NaN,33,NaN,NaN,1.0,NaN,NaN,NaN,0.683,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",2022-01-01,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
cleaned_titles[(cleaned_titles["genres"].isnull()) & (cleaned_titles["genres"].notnull())]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


可以看见，genres一项缺失的，其imdb_score也缺失

In [22]:
cleaned_titles[cleaned_titles["imdb_score"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945-01-01,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
75,tm132164,Bill Hicks: Sane Man,MOVIE,Sane Man was filmed before Bill recorded ‘Dang...,1989-01-01,R,80,comedy,US,NaN,NaN,NaN,NaN,3.377,7.5
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,documentation,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,family,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,reality,JP,12.0,NaN,NaN,NaN,7.730,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,2021-01-01,NaN,96,drama,NaN,NaN,tt20198164,NaN,NaN,NaN,NaN
5833,ts307884,HQ Barbers,SHOW,When a family run barber shop in the heart of ...,2021-01-01,TV-14,24,comedy,NG,1.0,NaN,NaN,NaN,0.840,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",2022-01-01,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
cleaned_titles.dropna(subset = ["genres"], axis = 0, inplace = True)
cleaned_titles[cleaned_titles["genres"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


In [24]:
cleaned_titles.dropna(subset = ["imdb_score"], axis = 0, inplace = True)

In [25]:
cleaned_titles[cleaned_titles["imdb_score"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


接下来查看是否有不一致数据

In [26]:
cleaned_titles.value_counts("genres")

genres
drama            3357
comedy           2419
thriller         1446
action           1339
romance          1080
crime            1066
documentation     981
family            769
animation         732
fantasy           727
european          679
scifi             647
horror            438
history           336
music             266
reality           226
war               221
sport             188
western            53
Name: count, dtype: int64

In [27]:
cleaned_titles.value_counts("production_countries")

production_countries
US    5648
IN    1610
GB    1068
JP    1046
FR     720
      ... 
AF       1
GT       1
GE       1
FO       1
CU       1
Name: count, Length: 108, dtype: int64

In [28]:
with pd.option_context('display.max_rows', None):
    print(cleaned_titles['production_countries'].value_counts())

production_countries
US         5648
IN         1610
GB         1068
JP         1046
FR          720
KR          637
ES          637
CA          608
DE          383
CN          295
MX          264
IT          224
BR          221
AU          217
TR          195
PH          192
AR          150
ID          149
BE          148
TW          133
NG          131
PL          126
ZA          103
NL          102
HK          102
CO           94
EG           93
DK           89
TH           87
SE           81
LB           70
NO           68
AE           52
IE           49
SG           47
XX           43
IL           42
RU           41
CL           35
CH           33
PS           32
BG           31
MY           30
SA           28
AT           28
IS           28
LU           27
NZ           27
PE           26
RO           25
QA           24
CZ           22
JO           19
FI           18
HU           18
UY           15
MA           15
PT           14
KH           10
KW           10
PR            9
PK 

"LB"和"Lebanon"指代同一个国家

In [29]:
cleaned_titles.replace({"Lebanon" : "LB"}, inplace = True)
with pd.option_context('display.max_rows', None):
    print(cleaned_titles.explode('production_countries')['production_countries'].value_counts())

production_countries
US    5648
IN    1610
GB    1068
JP    1046
FR     720
KR     637
ES     637
CA     608
DE     383
CN     295
MX     264
IT     224
BR     221
AU     217
TR     195
PH     192
AR     150
ID     149
BE     148
TW     133
NG     131
PL     126
ZA     103
HK     102
NL     102
CO      94
EG      93
DK      89
TH      87
SE      81
LB      71
NO      68
AE      52
IE      49
SG      47
XX      43
IL      42
RU      41
CL      35
CH      33
PS      32
BG      31
MY      30
AT      28
SA      28
IS      28
LU      27
NZ      27
PE      26
RO      25
QA      24
CZ      22
JO      19
FI      18
HU      18
UY      15
MA      15
PT      14
KW      10
KH      10
PK       9
PR       9
UA       8
MT       8
VN       8
LT       7
IR       7
CD       7
SU       7
TN       7
SN       6
AL       6
KE       6
GH       6
IQ       5
MU       5
CY       5
KN       4
GR       4
IO       4
SY       4
TZ       4
MC       4
GL       3
CM       3
AO       3
BS       3
HR       3
BD       3


# 整理数据

In [30]:
credits_with_titles = pd.merge(cleaned_credits, cleaned_titles, on="id", how="inner")

In [31]:
credits_with_titles.head()

,person_id,id,name,character,role,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
3,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
4,7064,tm84618,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179


只对演员进行分析

In [32]:
actor_with_titles = credits_with_titles.query('role == "ACTOR"')

In [33]:
actor_with_titles.head()

,person_id,id,name,character,role,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
3,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
4,7064,tm84618,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179


In [36]:
genres_person_id_imdb_scores = actor_with_titles.groupby(["genres","person_id"])["imdb_score"].mean().reset_index()
genres_person_id_imdb_scores

,genres,person_id,imdb_score
0,action,1000,6.866667
1,action,100007,7.000000
2,action,100013,6.400000
3,action,100019,6.500000
4,action,100020,6.500000
...,...,...,...
168876,western,993735,6.500000
168877,western,998673,7.300000
168878,western,998674,7.300000
168879,western,998675,7.300000


In [37]:
genres_max_imdb_score = genres_person_id_imdb_scores.groupby("genres")["imdb_score"].max()
genres_max_imdb_score

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

In [40]:
genres_max_imdb_score_person_id = pd.merge(genres_person_id_imdb_scores, genres_max_imdb_score, on = ["imdb_score","genres"], how = "inner")
genres_max_imdb_score_person_id

,genres,person_id,imdb_score
0,action,12790,9.3
1,action,1303,9.3
2,action,21033,9.3
3,action,336830,9.3
4,action,86591,9.3
...,...,...,...
131,war,826547,8.8
132,western,22311,8.9
133,western,28166,8.9
134,western,28180,8.9


In [41]:
actor_name_person_id = actor_with_titles[["person_id","name"]].drop_duplicates()
genres_max_imdb_score_person_id_name = pd.merge(genres_max_imdb_score_person_id, actor_name_person_id, on = "person_id",
                                                how = "left")
genres_max_imdb_score_person_id_name

,genres,person_id,imdb_score,name
0,action,12790,9.3,Olivia Hack
1,action,1303,9.3,Jessie Flower
2,action,21033,9.3,Zach Tyler
3,action,336830,9.3,André Sogliuzzo
4,action,86591,9.3,Cricket Leigh
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,22311,8.9,Koichi Yamadera
133,western,28166,8.9,Megumi Hayashibara
134,western,28180,8.9,Unsho Ishizuka
